In [ ]:
# Create reference images in EPSG:4326
# Note: Un-comment if you only have reference image in a different epsg
'''
!gdalwarp -of GTiff -t_srs EPSG:4326 -multi -wo NUM_THREADS=ALL_CPUS --config GDAL_CACHEMAX 16384 data/reference/ortofoto_reference_epsg32632.tif data/reference/ortofoto_reference_epsg4326.tif
!gdal_translate data/reference/ortofoto_reference_epsg4326.tif data/reference/ortofoto_reference_epsg4326_cog.tif -of COG -co COMPRESS=DEFLATE -co NUM_THREADS=ALL_CPUS -co BIGTIFF=YES --config GDAL_CACHEMAX 16384
!rm data/reference/ortofoto_reference_epsg4326.tif
!mv data/reference/ortofoto_reference_epsg4326_cog.tif data/reference/ortofoto_reference_epsg4326.tif

!gdalwarp -of GTiff -t_srs EPSG:4326 -multi -wo NUM_THREADS=ALL_CPUS --config GDAL_CACHEMAX 16384 data/reference/ortofoto_reference_epsg32632_nodata_mask.tif data/reference/ortofoto_reference_epsg4326_nodata_mask.tif
!gdal_translate data/reference/ortofoto_reference_epsg4326_nodata_mask.tif data/reference/ortofoto_reference_epsg4326_nodata_mask_cog.tif -of COG -co COMPRESS=DEFLATE -co NUM_THREADS=ALL_CPUS -co BIGTIFF=YES --config GDAL_CACHEMAX 16384
!rm data/reference/ortofoto_reference_epsg4326_nodata_mask.tif
!mv data/reference/ortofoto_reference_epsg4326_nodata_mask_cog.tif data/reference/ortofoto_reference_epsg4326_nodata_mask.tif
'''

In [ ]:
# Set the max. cache for gdal
!export GDAL_CACHEMAX=16384  # GDAL cache size in MB

In [ ]:
from arosics import COREG
from pathlib import Path
from tqdm import tqdm

# Find all images to be co-registered
im_targets = sorted(list(Path('/workspace/output_dir').glob('*.tif')))

# Do the co-registration
failed_im_targets = []
for im_target in tqdm(im_targets):
    # Define the destination path of the image after correction
    dst_path = Path('/workspace') / 'output_dir_coreg' / (im_target.stem + '_coreg.tif')
    
    # Set the path of the reference image to use for correcting
    reference_im_path = Path('/workspace/mnt/app/data/reference')
    im_reference = str(reference_im_path / 'ortofoto_reference_epsg4326.tif')
    
    print("")
    print("---------------------------------------")
    print(f"Co-registering {str(im_target)}")
    print(f"to reference image {str(im_reference)}")
    print(f"and saving output to {str(dst_path)}")
    if not dst_path.exists():
        # Run the co-registration algorithm using the Arosics package
        try:
            kwargs = {
            #     'mask_baddata_ref': str(reference_im_path / 'ortofoto_reference_epsg4326_nodata_mask.tif'),
                'path_out': str(dst_path),
                'fmt_out': 'COG',  # Output Cloud Optimized GeoTIFF (COG) format
                'out_crea_options': ['COMPRESS=DEFLATE', 'NUM_THREADS=ALL_CPUS', 'BIGTIFF=YES'],  # Use DEFLATE compression for the output geotiff
                'wp': (9.20, 56.10),  # Use a manually chosen window position
                'ws': (8192, 8192),  # Use a manually chosen window size
                'nodata': (255., -32768),  # Input/output nodata values
                'calc_corners': False,  # We provided the window position and size manually, so no reason to calculate corners
                'v': False,  # Verbose flag - all details for the calculations will be logged if True
                'q': True # Quiet flag - nothing will be logged if True
            }

            CR = COREG(str(im_reference), str(im_target), **kwargs)
            CR.calculate_spatial_shifts()
            CR.correct_shifts()
            print("Shift reliability is " + str(round(CR.shift_reliability, 2)) + "% for " + str(dst_path))
        except:
            print("")
            print(f"Error encountered with {str(im_target)}")
            failed_im_targets.append(im_target)
    else:
        print("")
        print(f"Image has already been co-registered and saved to {str(dst_path)}")
    print("---------------------------------------")
        
print("")
print("Following images were not co-registered:")
for failed_im_target in failed_im_targets:
    print(str(failed_im_target))

In [ ]:
# Try again for the failed ones, but set Arosics to verbose logging to see what the errors were
for im_target in tqdm(failed_im_targets):
    # Define the destination path of the image after correction
    dst_path = Path('/workspace') / 'output_dir_coreg' / (im_target.stem + '_coreg.tif')

    # Set the path of the reference image to use for correcting
    reference_im_path = Path('/workspace/mnt/app/data/reference')
    im_reference = str(reference_im_path / 'ortofoto_reference_epsg4326.tif')

    print("")
    print("---------------------------------------")
    print(f"Co-registering {str(im_target)}")
    print(f"to reference img {str(im_reference)}")
    print(f"and saving output to {str(dst_path)}")
    
    # Run the co-registration algorithm using the Arosics package
    try:
        kwargs = {
        #     'mask_baddata_ref': str(reference_im_path / 'ortofoto_reference_epsg4326_nodata_mask.tif'),
            'path_out': str(dst_path),
            'fmt_out': 'COG',  # Output Cloud Optimized GeoTIFF (COG) format
            'out_crea_options': ['COMPRESS=DEFLATE', 'NUM_THREADS=ALL_CPUS', 'BIGTIFF=YES'],  # Use DEFLATE compression for the output geotiff
            'wp': (9.20, 56.10),  # Use a manually chosen window position
            'ws': (8192, 8192),  # Use a manually chosen window size
            'nodata': (255., -32768),  # Input/output nodata values
            'calc_corners': False,  # We provided the window position and size manually, so no reason to calculate corners
            'v': True,  # Verbose flag - all details for the calculations will be logged if True
            'q': False # Quiet flag - nothing will be logged if True
        }

        CR = COREG(str(im_reference), str(im_target), **kwargs)
        CR.calculate_spatial_shifts()
        CR.correct_shifts()
        print("Shift reliability is " + str(round(CR.shift_reliability, 2)) + "% for " + str(dst_path))
    except:
        print("")
        print(f"Error encountered with {str(im_target)}")
    print("---------------------------------------")